In [235]:
# ============================================================
# DATA CLEANING & MERGING FOR PORT AUTHORITY BUS AND PASSENGER
# ============================================================

import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import re

tqdm.pandas()  # Enable progress bars for pandas operations

In [237]:
# ============================================================
# STEP 1: READ & CLEAN PABT BUS & PASSENGER FILES
# ============================================================

bus_file = 'All Recorded PABT Bus.txt'
pass_file = 'All Recorded PABT Passenger.txt'

# --- Read files
bus = pd.read_csv(bus_file, sep='\t')
passenger = pd.read_csv(pass_file, sep='\t')


# Function to aggressively clean carrier names for successful merging
def clean_carrier(s):
    """Removes spaces, punctuation, and converts to lowercase to ensure matching."""
    if pd.isna(s):
        return ''
    s = str(s).lower()
    s = s.replace(' ', '')  # Remove spaces (e.g., 'C & J Bus Lines' -> 'C&JBusLines')
    s = re.sub(r'[^a-z0-9]', '', s)  # Remove any non-alphanumeric characters (e.g., '-', '_', '&')
    return s

In [239]:
# --- Strip column names & whitespace
bus.columns = bus.columns.str.strip()
passenger.columns = passenger.columns.str.strip()

# Strip whitespace from all string data columns
bus = bus.apply(lambda col: col.str.strip() if col.dtype == 'object' else col)
passenger = passenger.apply(lambda col: col.str.strip() if col.dtype == 'object' else col)

In [241]:
# Normalize date columns. This step is critical as formats differ:
# - BUS FILE dates are DD/MM/YYYY (e.g., '08/02/2021'), requiring dayfirst=True.
bus['Start_Date'] = pd.to_datetime(bus['Start_Date'], errors='coerce', dayfirst=True)
bus['End_Date'] = pd.to_datetime(bus['End_Date'], errors='coerce', dayfirst=True)

# - PASSENGER FILE dates include a time component (e.g., '2020-12-07 00:00:00.000').
#   We convert to datetime, then extract only the date component to align with the bus data.
passenger['Start_Date'] = pd.to_datetime(passenger['Start_Date'], errors='coerce').dt.date
passenger['End_Date'] = pd.to_datetime(passenger['End_Date'], errors='coerce').dt.date


In [243]:
# --- Convert Volume to numeric (handle NULLs/empty strings)
bus['Volume'] = pd.to_numeric(bus['Volume'].replace(['NULL',''], np.nan), errors='coerce')
passenger['Volume'] = pd.to_numeric(passenger['Volume'].replace(['NULL',''], np.nan), errors='coerce')

In [245]:
# Apply the carrier cleaning function to both dataframes
bus['Carrier_clean'] = bus['Carrier'].apply(clean_carrier)
passenger['Carrier_clean'] = passenger['Carrier'].apply(clean_carrier)

In [247]:
# Ensure the bus date columns are converted to match the passenger date type (datetime.date)
bus['Start_Date'] = pd.to_datetime(bus['Start_Date']).dt.date
bus['End_Date'] = pd.to_datetime(bus['End_Date']).dt.date

In [249]:
# --- Optional: filter out early Dec 2020 from Passenger to match Bus
# passenger = passenger[passenger['Start_Date'] >= pd.to_datetime('2021-02-08').date()]

In [251]:
bus['Carrier_clean'] = bus['Carrier'].apply(clean_carrier)
passenger['Carrier_clean'] = passenger['Carrier'].apply(clean_carrier)

In [253]:
# Count matched & unmatched
combined_df = pd.merge(
    bus, passenger,
    left_on=['Start_Date', 'End_Date', 'Carrier_clean'],
    right_on=['Start_Date', 'End_Date', 'Carrier_clean'],
    how='outer',
    suffixes=('_Bus', '_Passenger'),
    indicator=True # Adds '_merge' column to show match status
)

# Check the success of the merge
matched_count = (combined_df['_merge'] == 'both').sum()
print(f"Total Matched Rows: {matched_count}")

Total Matched Rows: 2772


In [255]:
# After merging bus_df and passenger_df into combined_df
combined_df[['Volume_Bus', 'Volume_Passenger']] = combined_df[['Volume_Bus', 'Volume_Passenger']].fillna(0)

In [257]:
# ===============================
# Save combined bus-passenger dataset
# ===============================

# OUTPUT 1: Full Merged Data (All Columns) ---
combined_df.to_csv('Raw_Merged_PABT_Bus_Passenger.csv', index=False)

In [259]:
# OUTPUT 2: Final Output ---

# Define the final requested columns
reduced_cols = ['Start_Date', 'End_Date', 'Carrier_clean', 'Volume_Bus', 'Volume_Passenger']

# Select only the requested columns from the full combined DataFrame
full_ready_to_use = combined_df[reduced_cols]

# Save the final, reduced-column file
output_file_reduced = 'data/output/Merged_PABT_Bus_Passenger_For_PowerBI.csv'
full_ready_to_use.to_csv(output_file_reduced, index=False)

print(f"\nFinal merged data (2,796 records, 4 columns) saved to: {output_file_reduced}")


Final merged data (2,796 records, 4 columns) saved to: data/output/Merged_PABT_Bus_Passenger_For_PowerBI.csv


In [265]:
# This is only required if we want to perform predictive analysis on PABT BUS and Passenger dataset.

import pandas as pd
import numpy as np
from scipy import stats

# Load merged PABT data
df = pd.read_csv("data/Output/Merged_PABT_Bus_Passenger_For_PowerBI.csv", parse_dates=["Start_Date","End_Date"])

# 1. Log transform (add 1 to avoid log(0))
df["log_Volume_Bus"] = np.log1p(df["Volume_Bus"])
df["log_Volume_Passenger"] = np.log1p(df["Volume_Passenger"])

# 2. Outlier capping at the 99th percentile
for col in ["Volume_Bus","Volume_Passenger"]:
    upper = df[col].quantile(0.99)
    df[f"{col}_capped"] = np.where(df[col] > upper, upper, df[col])

# 3. Zero-inflation flag
df["bus_zero"] = (df["Volume_Bus"] == 0).astype(int)
df["pass_zero"] = (df["Volume_Passenger"] == 0).astype(int)

# 4. Carrier-specific normalization (z-score within each carrier)
df[["bus_z","pass_z"]] = df.groupby("Carrier_clean")[["Volume_Bus","Volume_Passenger"]].transform(
    lambda x: (x - x.mean())/ x.std(ddof=0)
)

# 5. (Optional) Zero-inflated Poisson features
#    e.g., probability of zero vs non-zero per carrier
carrier_zero_rates = df.groupby("Carrier_clean")["bus_zero"].mean().rename("bus_zero_rate")
df = df.join(carrier_zero_rates, on="Carrier_clean")

# Save prepared dataset
df.to_csv("data/output/Merged_PABT_Bus_Passenger_For_Predictive.csv", index=False)

# Now df contains:
#  - log_Volume_Bus, log_Volume_Passenger
#  - Volume_Bus_capped, Volume_Passenger_capped
#  - bus_zero, pass_zero flags
#  - bus_z, pass_z (carrier-normalized)
#  - bus_zero_rate per carrier
